<a href="https://colab.research.google.com/github/fredjgermain/plant_recon/blob/main/plant_recon_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Before recloning from github use "Disconnect and delete runtime" 
!git clone https://github.com/fredjgermain/plant_recon.git 

# Refresh browser or wait a bit to see cloned folder appear in Colab 
# Once "/content/plant_recon" is done cloning in Colab 
%cd /content/plant_recon 
%ls 
!git status 

Cloning into 'plant_recon'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.


# PLANT RECOGNITION
*Description*: Train a DL model to recon plants, trees, fruits, pests.

## TODO

[!] Find some test images 

## PROJECT SETTING 


In [ ]:
# Import libraries 
import sys 
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split 
from matplotlib.pyplot import imread 

import tensorflow as tf 
import tensorflow_hub as hub 

from IPython.display import Image 
import datetime 
import json 

# # Load tensorboard notebook extension
%load_ext tensorboard

# Print libraries versions 
print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
print("Tensor flow version:", tf.__version__)
print("Hub version:", hub.__version__)

# Check for GPU -----------------------------------------
print("GPU availability:", 
      "GPU is available" if tf.config.list_physical_devices("GPU") 
      else "No GPU available") 

# Projects paths and constants --------------------------
PROJECT_PATH = "/content/drive/MyDrive/plant_recon"
MODEL_LOG_FILE = f'{PROJECT_PATH}/models/modeling_log.json'

DEFAULT_IMG_SIZE = 224
DEFAULT_BATCH_SIZE = 32

SAMPLE_SUFFIX="model_sample"
FULL_SUFFIX="model_full"

SAMPLE_SIZE = 500


## FUNCTIONS

In [ ]:
# Helper functions =============================================================
# Create a time stamped string to now. 
def time_stamp() -> str: 
  return datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S")

# Func to plot image data batch 
def show_N_images(images, unique_labels, labels, N=25):
  plt.figure( figsize=(10,10) )
  for i in range(25): 
    ax = plt.subplot(5,5,i+1) 
    plt.imshow(images[i]) 
    plt.title( unique_labels[labels[i].argmax()] ) # datas == breeds 
    plt.axis("off") 


# Array of P, images, integer
def plot_pred(categories, predictions, labels, images, n=1):
  P, label, image = predictions[n], labels[n], images[n]
  pred_label = get_predicted_label(categories, P)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  if pred_label == label:
    color = "green"
  else:
    color = "red"
  
  plt.title("{} {:2.0f}% {} ".format( pred_label, np.max(P)*100, label), color=color)


def plot_pred_conf(categories, predictions, labels, n=1): 
  Ps, label = predictions[n], labels[n]
  pred_label = get_predicted_label(categories, Ps)
  top_10_indexes = Ps.argsort()[-10:][::-1]
  top_10_Ps = Ps[top_10_indexes]
  top_10_predicted_labels = categories[top_10_indexes] # datas == breeds 
  top_plot = plt.bar(np.arange( len(top_10_predicted_labels)), top_10_Ps, color="grey")
  
  plt.xticks( 
      np.arange(len(top_10_predicted_labels) ), 
      labels=top_10_predicted_labels, 
      rotation="vertical") 
  
  if np.isin(label, top_10_predicted_labels):
    top_plot[np.argmax(top_10_predicted_labels == label)].set_color("green")
  else:
    pass



# DATA BATCHING & IMG PROCESSING ==============================================
# Create a function for preprocessing images 
"""
Takes an images path and returns that images as a tensor. 
"""
def process_image(image_path, image_size=DEFAULT_IMG_SIZE): 
  # Read image
  image = tf.io.read_file(image_path) 
  # Turn jpeg into numerical Tensor with 3 color channels (Red, Green, Blue) 
  image = tf.image.decode_jpeg(image, channels=3) 
  # Normalize image, convert the color channel values from 0-255 to 0-1 values 
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize image to our desired value (IMG_SIZE, IMG_SIZE) 
  image = tf.image.resize(image, size=[image_size, image_size]) 
  return image 

# Create a function returning a tuple (image, label) 
def get_image_label(image_path, label):
  """
  Returns a pair "tensor and label" from a image path and a string label. 
  """
  image = process_image(image_path)
  return image, label


# Turning data into Batches
"""
Purpose of batches: 
GPU has limited memory and thus can only processus a limited number of images at once. 
"""

# Makes batches size 32 as default
def create_data_batches(X, y=None, data_type="training", batch_size=DEFAULT_BATCH_SIZE): 
  """ 
  Create batches of data out of image (X) and label (y) pairs. 
  Shuffles training data, 
  DOES NOT shuffle validation data. 
  """ 
  # data_type = "test" if test_data else data_type = "validation" if valid_data else data_type = "training" 
  print(f'Creating {data_type} data batches {batch_size}') 

  if data_type == "testing": 
    data = tf.data.Dataset.from_tensor_slices( (tf.constant(X)) ) 
    process_func = process_image 
  else: 
    data = tf.data.Dataset.from_tensor_slices( (tf.constant(X), tf.constant(y)) ) 
    process_func = get_image_label 
    if data_type == "training": 
      data = data.shuffle(buffer_size=len(X)) 
  return data.map(process_func).batch(batch_size) 

## from numpy.ma.core import append
# Unbatch validation_batch data 
def unbatch(categories, batch_data): 
  images = [] 
  labels = [] 
  for image, label in batch_data.unbatch().as_numpy_iterator(): 
    images.append(image) 
    labels.append(categories[np.argmax(label)]) 
  return images, labels 

# unbatch data without labels 
def unbatch_without_labels(batch_data): 
  images = [] 
  for image in batch_data.unbatch().as_numpy_iterator(): 
    images.append(image) 
  return images 



# MODEL CREATION ===============================================================
# See TensorFlow Hub ... (was in development back then, thus it might be different now) 
# See architecture "mobilenet" 
# src: "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5" 
def create_model(input_shape, output_shape, model_url):
  print("Building model with:", model_url) 
  # Setup model layers 
  model = tf.keras.Sequential([
    hub.KerasLayer(model_url), 
    tf.keras.layers.Dense(units=output_shape, activation="softmax")
  ]) 
  # Compile model 
  model.compile(
      loss = tf.keras.losses.CategoricalCrossentropy(), 
      optimizer = tf.keras.optimizers.Adam(), 
      metrics = ["accuracy"]
  )
  # Build model
  model.build(input_shape) 
  return model 


# Function to train a model 
# callbacks = [tensorboard, early_stopping] 
def train_model(model, training_batch, validation_batch, epochs, callbacks): 
  # create model
  # model = create_model(input_shape, output_shape, model_url) 
  # tensorboard = create_tensorboard_callback(project_path) 
  model.fit(x=training_batch, 
            epochs = epochs, 
            validation_data = validation_batch, 
            validation_freq = 1, 
            callbacks = callbacks) 
  return model


# Get predicted label ..........................................................
def get_predicted_label(categories, predictions):
  maxcol = np.argmax(predictions)
  return categories[maxcol] # datas == breeds

# Visualize prediction .........................................................
def visualize_predictions(categories, predictions, labels, images):
  i_m = 0
  nr = 3
  nc = 2
  nimages = nr*nc
  plt.figure(figsize=(10*nc, 5*nr))
  for i in range(nimages):
    plt.subplot(nr, 2*nc, 2*i+1) 
    plot_pred(categories=categories, 
              predictions=predictions, 
              labels=labels, 
              images=images, 
              n=i+i_m)
    plt.subplot(nr, 2*nc, 2*i+2) 
    plot_pred_conf(categories=categories, 
              predictions=predictions, 
              labels=labels, 
              n=i+i_m)
  plt.tight_layout(h_pad=1.0)
  plt.show()


# Creating callbacks 
# Tensorboad callback ..........................................................
def create_tensorboard_callback(project_path):
  """ 
    Functions to either 
    - Monitor models progress 
    - Save models progress 
    - Interupt models training to prevent overt fitting 
  """ 
  # Create logs 
  logdir = os.path.join( f'{project_path}/logs', time_stamp() )
  return tf.keras.callbacks.TensorBoard(logdir)



# SAVING MODEL =================================================================
def save_model(project_path, model, suffix=None): 
  modeldir = os.path.join(f"{project_path}/models", time_stamp())
  model_name = f"-{suffix}.h5"
  print(f"Saving model to: {model_name} at {modeldir} ")
  model.save(f"{modeldir}{model_name}")
  return f"{modeldir}{model_name}"

def load_model(model_path): 
  print(f"Loading model from: {model_path}")
  model = tf.keras.models.load_model( model_path, custom_objects={"KerasLayer":hub.KerasLayer}) 
  return model



# Read/Write model_log =========================================================
def read_model_log(model_log_file):
  return json_to_dict(model_log_file)

def write_model_log(model_log_file, model_log):
  dict_to_json(model_log_file, model_log)


# JSON helper function 
def dict_to_json(jsonfile, dict): 
  with open(jsonfile, "wb") as f: 
    f.write(json.dumps(dict).encode("utf-8")) 
  return jsonfile

def json_to_dict(jsonfile):
  with open(jsonfile, "r") as f:
    data = json.load(f)
  return data

## DATA WRANGLING

### Data reading

In [ ]:
# Read labels csv 
# Contains couple of image id and label. 
labels_csv = pd.read_csv(f'{PROJECT_PATH}/data/data.csv') 

# divides csv into its 2 columns
filenames = [f'{PROJECT_PATH}/data/img/{filename}.jpg' for filename in labels_csv["id"]] 
labels = labels_csv["label"]

# list unique labels (strings) 
categories = np.unique(labels) 

# list of filenames 
X = filenames 
# list of list of targets (image-file corresponds to labels? bool) 
y = [label == categories for label in labels] 

### Data summary

In [ ]:
# Value count per categories, median, min, max value count
categories_counts = labels.value_counts().sort_values()
N_categories = len(categories)

categories_counts_median = categories_counts.median() 
categories_counts_max = categories_counts.max() 
categories_counts_min = categories_counts.min() 

print(categories) 
print("Number of different categories:", N_categories) 
print("Median N images per breed: ", categories_counts.median() ) 

print("Minimum N images per breed: ", categories_counts.values[0] ) 
print("Categories with least frequencies:", categories_counts.index[0]) 
print("Maximum N images per breed: ", categories_counts.values[N_categories-1]) 
print("Categories with most frequencies:", categories_counts.index[N_categories-1]) 

### Data sampling, splitting and batching training and validation data

In [ ]:
"""
  possible? : 
    create_batches before sampling, 
    sampling on the batch itself, 
    train sample model using a sample of batched data ? 
"""


# SAMPLE DATASET ----------------------------------------
# sampling
sample_X = X[:SAMPLE_SIZE]
sample_y = y[:SAMPLE_SIZE]
# splitting
sample_X_train, sample_X_val, sample_y_train, sample_y_val = train_test_split(
  sample_X, sample_y, 
  test_size=0.2, 
  random_state=42 
) 
# batching 
sample_training_batch = create_data_batches( 
  sample_X_train, sample_y_train) 
sample_validation_batch = create_data_batches( 
  sample_X_val, sample_y_val, data_type="validation") 


# FULL DATASET --------------------------------------------------
# splitting
full_X_train, full_X_val, full_y_train, full_y_val = train_test_split(
  X, y, 
  test_size=0.2, 
  random_state=42 
)
# batching 
full_training_batch = create_data_batches( 
  full_X_train, full_y_train) 
full_validation_batch = create_data_batches( 
  full_X_val, full_y_val, data_type="validation") 



## MODEL TRAINING

In [ ]:
# MODELING CONSTANTS
INPUT_SHAPE = [None, DEFAULT_IMG_SIZE, DEFAULT_IMG_SIZE, 3] 
OUTPUT_SHAPE = len(categories) 
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5" 

# NUM_EPOCHS = 100 #@param {type:"slider", min:10, max:100}
EPOCHS = 100 

# Callback function
tensorboard = create_tensorboard_callback(PROJECT_PATH)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3) 
CALLBACKS = [tensorboard, early_stopping] 

# Check for GPU availability
print("GPU availability:", 
      "GPU is available" if tf.config.list_physical_devices("GPU") 
      else "No GPU available") 

### Sample model

In [ ]:
# # Create sample model
# sample_model = create_model(INPUT_SHAPE, OUTPUT_SHAPE, MODEL_URL) 
# sample_model.summary() 

# # Train sample model 
# sample_model = train_model( 
#     model=sample_model, 
#     training_batch=sample_training_batch, 
#     validation_batch=sample_validation_batch, 
#     epochs= EPOCHS, callbacks=CALLBACKS) 

# # Save sample model
# sample_model_path = save_model(PROJECT_PATH, sample_model, SAMPLE_SUFFIX)

# # Log sample model 
# model_log = read_model_log(MODEL_LOG_FILE)
# model_log[SAMPLE_SUFFIX] = f"{sample_model_path}"
# write_model_log(MODEL_LOG_FILE, model_log)

### Full model

In [ ]:
# Create model
full_model = create_model(INPUT_SHAPE, OUTPUT_SHAPE, MODEL_URL) 
full_model.summary() 

# Train model 
full_model = train_model( 
    model=full_model, 
    training_batch=full_training_batch, 
    validation_batch=full_validation_batch, 
    epochs= EPOCHS, callbacks=CALLBACKS) 

# Save model
full_model_path = save_model(PROJECT_PATH, full_model, FULL_SUFFIX)

# Log model 
model_log = read_model_log(MODEL_LOG_FILE)
model_log[FULL_SUFFIX] = f"{full_model_path}"
write_model_log(MODEL_LOG_FILE, model_log)

## MODEL EVALUATION

In [ ]:
## Tensorboard logs 
## [!!] This bit will not work if browser blocks cookies. [!!] 
log_base_dir = f"{PROJECT_PATH}/logs" 
%tensorboard --logdir /content/drive/MyDrive/plant_recon/logs 
# --port=6006 

### Predictions

In [ ]:
# EVALUATE SAMPLE MODEL 
last_saved_model_path = read_model_log(MODEL_LOG_FILE)[FULL_SUFFIX]
loaded_model = load_model(last_saved_model_path)
# evaluation_training_batch = sample_training_batch
evaluation_validation_batch = full_validation_batch

predictions = loaded_model.predict(evaluation_validation_batch) 

# atrow = 42 
# maxcol = np.argmax(predictions[atrow]) 
# print( f"Max P at index: {np.max(predictions[atrow])}" ) 
# print( f"Sum of P at index {atrow}: {np.sum(predictions[atrow]) }" ) # ~1 sum of probabilities 
# print( f"Max index: {maxcol} ") 
# print( f"Predicted label: {categories[maxcol]} ") 
# print(get_predicted_label(categories, predictions[81]))

images, labels = unbatch(categories, evaluation_validation_batch) 
plot_pred(categories, predictions, labels, images, n=77) 

visualize_predictions(categories, predictions, labels, images) 


## MODEL TESTING

In [ ]:
# load saved model
last_saved_model_path = read_model_log(MODEL_LOG_FILE)[FULL_SUFFIX]
loaded_model = load_model(last_saved_model_path)

# read test data from filenames in test folder 
test_filenames = [f'{PROJECT_PATH}/data/test/{filename}' for filename in os.listdir(f'{PROJECT_PATH}/data/test') ] 
X = test_filenames

test_batched = create_data_batches(X, data_type="testing")
test_unbatched = unbatch_without_labels(test_batched)

predictions = loaded_model.predict(test_batched, verbose=1)
predictions_labels = [get_predicted_label(categories, predictions[i]) for i in range(len(predictions))]

plt.figure(figsize=(10,10))
for i, image in enumerate(test_unbatched):
  plt.subplot(1, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.title(predictions_labels[i])
  plt.imshow(image)
